<a href="https://colab.research.google.com/github/jj-tech-io/Modify_Texture_Docker/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
from pathlib import Path
import subprocess
import os

drive.mount('/content/drive', force_remount=True)
!rm -rf '/content/drive/MyDrive/Colab_Projects/Modify_Texture_Notebook'

# Define the name of your project folder
project_folder_name = "Modify_Texture_Notebook"

# Define the path in Google Drive
drive_project_path = Path('/content/drive/MyDrive/Colab_Projects') / project_folder_name

# Clone the Git repository (replace 'your_repo_url' with your actual repository URL)
repo_url = 'https://github.com/jj-tech-io/Modify_Texture_Docker.git'  # <-- Replace with your repository URL
!git clone {repo_url} {drive_project_path}

# Function to install requirements from requirements.txt if it exists
def install_requirements(repo_path):
    requirements_file = repo_path / 'requirements.txt'
    if requirements_file.is_file():
        subprocess.run(['pip', 'install', '-r', str(requirements_file)], check=True)
        print("Requirements installed successfully.")
    else:
        print("requirements.txt not found in the project folder.")

# Run the function to install any requirements
install_requirements(drive_project_path)
#upload all files from repo to colab
!cp -r {drive_project_path}/* .

# Replace with your textures folder path inside Google Drive
textures_directory = '/content/drive/My Drive/textures'


In [15]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import importlib
import sys
import time
import tkinter as tk
from tkinter import ttk
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import io
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display
display_image = lambda x: display(Image.fromarray(x.astype(np.uint8)))
import CONFIG

importlib.reload(CONFIG)
sys.path.append(r"AE_Inference")
import AE_Inference
from AE_Inference import encode, decode, age_mel, age_hem
importlib.reload(AE_Inference)


class SkinParameterAdjustmentApp:
    def __init__(self, image, mel_aged, oxy_aged, skin, face):
        self.skin = cv2.resize(skin, (512, 512))
        self.face = cv2.resize(face, (512, 512))
        self.original_label = None
        self.modified_label = None
        self.encoder = None
        self.decoder = None
        self.image = image
        self.mel_aged = mel_aged
        self.oxy_aged = oxy_aged
        self.WIDTH = 512
        self.HEIGHT = 512
        self.load_models()
        self.load_images()


    def load_models(self):
        self.decoder = load_model(CONFIG.DECODER_PATH)
        self.encoder = load_model(CONFIG.ENCODER_PATH)

    def load_images(self):
        original_image = None
        mel_aged = None
        oxy_aged = None
        if type(self.image) == str:
            original_image = cv2.imread(self.image, cv2.IMREAD_COLOR)
        else:
            original_image = self.image
        if type(self.mel_aged) == str:
            mel_aged = cv2.imread(self.mel_aged, cv2.IMREAD_GRAYSCALE)
        else:
            mel_aged = self.mel_aged
        if type(self.oxy_aged) == str:
            oxy_aged = cv2.imread(self.oxy_aged, cv2.IMREAD_GRAYSCALE)
        else:
            oxy_aged = self.oxy_aged
        if self.image is None or self.mel_aged is None or self.oxy_aged is None:
            print(f"Error: read [image, mel_aged, oxy_aged] = [{self.image}, {self.mel_aged}, {self.oxy_aged}]")
            sys.exit()

        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB).astype(np.float32)

        original_4k = original_image.copy().astype(np.float32)
        modified_4k = original_4k.copy().astype(np.float32)
        original_image = cv2.resize(original_image, (self.WIDTH, self.HEIGHT))
        modified_image = original_image.copy()
        mel_aged = cv2.resize(mel_aged, (self.WIDTH, self.HEIGHT))
        oxy_aged = cv2.resize(oxy_aged, (self.WIDTH, self.HEIGHT))
        oxy_aged = cv2.bitwise_not(oxy_aged)
        oxy_aged = cv2.GaussianBlur(oxy_aged, (15, 15), 0)
        oxy_aged = oxy_aged / np.max(np.abs(oxy_aged))
        oxy_aged *= 0.1

        mel_aged = mel_aged.reshape(-1, )
        oxy_aged = oxy_aged.reshape(-1, )
        parameter_maps_original, encode_time = encode(original_image.reshape(-1, 3) / 255.0)

        self.original_image = original_image
        self.modified_image = modified_image
        self.mel_aged = mel_aged
        self.oxy_aged = oxy_aged
        self.parameter_maps = parameter_maps_original.copy()
        self.parameter_maps_original = parameter_maps_original
        self.parameter_maps_original, self.encode_time = encode(self.original_image.reshape(-1, 3) / 255.0)
        self.original_4k = cv2.resize(original_4k, (self.WIDTH, self.WIDTH))
        self.modified_4k = cv2.resize(modified_4k, (self.WIDTH, self.WIDTH))
        self.mel_aged_4k = cv2.resize(mel_aged.reshape((self.WIDTH, self.HEIGHT)), (self.WIDTH, self.WIDTH))
        self.oxy_aged_4k = cv2.resize(oxy_aged.reshape((self.WIDTH, self.HEIGHT)), (self.WIDTH, self.WIDTH))
        self.parameter_maps_original_4k, self.encode_time_4k = encode(self.original_4k.reshape(-1, 3) / 255.0)

    def create_slider(self, description, min, max, step, value):
        return widgets.FloatSlider(description=description, min=min, max=max, step=step, value=value)

    def save_4k_image(self):
        self.WIDTH = 4096
        self.HEIGHT = 4096
        self.load_images()
        parameter_maps_original = self.parameter_maps_original_4k.copy()
        parameter_maps = self.parameter_maps_original_4k.copy()
        age_coef = self.age_coef_slidervalue
        scale_c_m = self.cm_slidervalue
        scale_c_h = self.ch_slidervalue
        scale_b_m = self.bm_slidervalue
        scale_b_h = self.bh_slidervalue
        scale_t = self.t_slidervalue
        cm_mask_slider = self.cm_mask_slidervalue
        bh_mask_slider = self.bh_mask_slidervalue
        global_scaling_maps = self.global_scaling_maps_slidervalue
        global_scaling_masks = self.global_scaling_masks_slider.value
        parameter_maps[:, 0] = age_mel(parameter_maps[:, 0], age_coef)
        parameter_maps[:, 1] = age_hem(parameter_maps[:, 1], age_coef)
        parameter_maps[:, 0] = scale_c_m* parameter_maps[:, 0]
        parameter_maps[:, 1] = scale_c_h* parameter_maps[:, 1]
        parameter_maps[:, 2] = scale_b_m*parameter_maps[:, 2]
        parameter_maps[:, 3] = scale_b_h*parameter_maps[:, 3]
        parameter_maps[:, 4] = scale_t*parameter_maps[:, 4]
        cm_new =  (cm_mask_slider * self.mel_aged.reshape(-1)) + (1 - cm_mask_slider) * parameter_maps[:, 0]
        parameter_maps[:, 0] = cm_new
        parameter_maps[:, 1] = (bh_mask_slider * self.oxy_aged.reshape(-1)) + parameter_maps[:, 1]

        recovered, decode_time = decode(parameter_maps)
        recovered = np.asarray(recovered).reshape((self.WIDTH, self.HEIGHT, 3)) * 255

        # Define the file path in Google Drive where you want to save the image
        file_path = "/content/drive/My Drive/your_directory/modified.png"

        # Save the image using OpenCV
        cv2.imwrite(file_path, cv2.cvtColor(recovered, cv2.COLOR_BGR2RGB))
        self.WIDTH = 512
        self.HEIGHT = 512
        self.load_images()

    def update_plot(self, changed_slider=None):
        parameter_maps_original = self.parameter_maps_original.copy()
        parameter_maps = self.parameter_maps_original.copy()
        age_coef = self.age_coef_slider.value
        scale_c_m = self.cm_slider.value
        scale_c_h = self.ch_slider.value
        scale_b_m = self.bm_slider.value
        scale_b_h = self.bh_slider.value
        scale_t = self.t_slider.value
        cm_mask_slider = self.cm_mask_slider.value
        bh_mask_slider = self.bh_mask_slider.value
        global_scaling_maps = self.global_scaling_maps_slider.value
        global_scaling_masks = self.global_scaling_masks_slider.value
        parameter_maps[:, 0] = age_mel(parameter_maps[:, 0], age_coef)
        parameter_maps[:, 1] = age_hem(parameter_maps[:, 1], age_coef)
        parameter_maps[:, 0] = scale_c_m* parameter_maps[:, 0]
        parameter_maps[:, 1] = scale_c_h* parameter_maps[:, 1]
        parameter_maps[:, 2] = scale_b_m*parameter_maps[:, 2]
        parameter_maps[:, 3] = scale_b_h*parameter_maps[:, 3]
        parameter_maps[:, 4] = scale_t*parameter_maps[:, 4]
        if changed_slider == 'cm_mask':
            print(f"cm_mask: {cm_mask_slider}")
        cm_new =  (cm_mask_slider * self.mel_aged.reshape(-1)) + (1 - cm_mask_slider) * parameter_maps[:, 0]
        # bm_new = (cm_mask_slider * self.mel_aged.reshape(-1)) + (1 - cm_mask_slider) * parameter_maps[:, 2]
        parameter_maps[:, 0] = cm_new
        # parameter_maps[:, 3] = bm_new
        if changed_slider == 'bh_mask':
            print(f"bh_mask: {bh_mask_slider}")
        parameter_maps[:, 3] = (bh_mask_slider * self.oxy_aged.reshape(-1)) + (1 - bh_mask_slider)*parameter_maps[:, 3]

        recovered, decode_time = decode(parameter_maps)
        recovered = np.asarray(recovered).reshape((self.WIDTH, self.HEIGHT, 3)) * 255
        self.parameter_maps = parameter_maps
        self.modified_image = recovered
        self.update_images(self.original_image, self.modified_image)

        return recovered

    def update_images(self, original, modified):
        # Convert the images to PIL format if not already
        original_pil = Image.fromarray(np.uint8(original)) if not isinstance(original, Image.Image) else original
        modified_pil = Image.fromarray(np.uint8(modified)) if not isinstance(modified, Image.Image) else modified

        # Convert PIL image to a byte array in PNG format
        buffer_original = io.BytesIO()
        buffer_modified = io.BytesIO()
        original_pil.save(buffer_original, format="PNG")
        modified_pil.save(buffer_modified, format="PNG")

        # Create ipywidget images from the byte array data
        original_widget = widgets.Image(value=buffer_original.getvalue(), format='png', width=512, height=512)
        modified_widget = widgets.Image(value=buffer_modified.getvalue(), format='png', width=512, height=512)

        # Place the image widgets side by side in an HBox
        hbox = widgets.HBox([original_widget, modified_widget])

        # Display the HBox
        display(hbox)

    def create_gui(self):
        # Sliders
        self.age_coef_slider = self.create_slider("Age(decades):", 0, 10, 0.1, 0.0)
        self.cm_slider = self.create_slider("Cm:", 0, 2, 0.1, 1)
        self.ch_slider = self.create_slider("Ch:", 0, 2, 0.1, 1)
        self.bm_slider = self.create_slider("Bm:", 0, 2, 0.1, 1)
        self.bh_slider = self.create_slider("Bh:", 0, 2, 0.1, 1)
        self.t_slider = self.create_slider("T:", 0, 2, 0.1, 1)
        self.cm_mask_slider = self.create_slider("Melanin Mask:", -1, 1, 0.1, 0)
        self.bh_mask_slider = self.create_slider("Oxy-Hb Mask:", -1, 1, 0.1, 0)
        self.global_scaling_maps_slider = self.create_slider("Map(s):", 0, 2, 0.1, 1.0)
        self.global_scaling_masks_slider = self.create_slider("Mask(s):", 0, 2, 0.1, 1.0)

        # Button
        self.save_button = widgets.Button(description="Save 4K Image")

        # Image display area
        image_display = widgets.Output()

        # Callbacks
        def on_slider_change(change):
            with image_display:
                clear_output(wait=True)
                # Logic to update image based on slider values
                self.update_plot(change)
                # self.display(self.modified_image)

        def on_save_button_clicked(button):
            self.save_4k_image(button)

        # Setting the observers for the widgets
        self.age_coef_slider.observe(on_slider_change, names='value')
        self.cm_slider.observe(on_slider_change, names='value')
        self.ch_slider.observe(on_slider_change, names='value')
        self.bm_slider.observe(on_slider_change, names='value')
        self.bh_slider.observe(on_slider_change, names='value')
        self.t_slider.observe(on_slider_change, names='value')
        self.cm_mask_slider.observe(on_slider_change, names='value')
        self.bh_mask_slider.observe(on_slider_change, names='value')
        self.global_scaling_maps_slider.observe(on_slider_change, names='value')
        self.global_scaling_masks_slider.observe(on_slider_change, names='value')
        self.save_button.on_click(on_save_button_clicked)

        # Layout
        sliders = widgets.VBox([self.age_coef_slider, self.cm_slider, self.ch_slider, self.bm_slider, self.bh_slider, self.t_slider, self.cm_mask_slider, self.bh_mask_slider, self.global_scaling_maps_slider, self.global_scaling_masks_slider])
        gui = widgets.VBox([sliders, self.save_button, image_display])

        display(gui)

    def run(self):
        self.create_gui()


Num GPUs Available:  0


In [ ]:
import os
from pathlib import Path
from tkinter import *
from tkinter.ttk import *
import importlib
import sys

import morph
from morph import *
import AE_Inference
from AE_Inference import encode, decode, age_mel, age_hem, get_masks
import segmentation
from segmentation import *
importlib.reload(segmentation)
importlib.reload(morph)

WIDTH = 512
HEIGHT = 512

# morph the source image to the target image
def morph_images(example_image_path, target_image_path):
    # Read the images into NumPy arrays
    target_image = cv2.imread(target_image_path.as_posix())
    example_image = cv2.imread(example_image_path.as_posix())
    # Check if the images are read correctly
    if example_image is None:
        print(f"Error: could not read source image {example_image_path}")
        sys.exit()
    if target_image is None:
        print(f"Error: could not read target image {target_image_path}")
        sys.exit()
    # Resize the images
    target_image = cv2.resize(target_image, (WIDTH, HEIGHT))
    example_image = cv2.resize(example_image, (WIDTH, HEIGHT))
    landmarks1 = get_landmarks(example_image)
    landmarks2 = get_landmarks(target_image)
    warped_example_image, delaunay, transformation_matrices = warp_image(example_image, target_image,
        landmarks1, landmarks2)
    return warped_example_image, target_image, example_image

# extract masks from source
def extract_masks(image):
    Cm, Ch, Bm, Bh, T = get_masks(image)
    landmarks = get_landmarks(image)
    #inverted masks
    combined_mask, lips, eyes, nose, eye_bags, face,oxy_mask, landmark_object, av_skin_color = create_combined_mask(image)
    skin = threshold_face_skin_area(image,av_skin_color,mask=combined_mask)
    return Cm, Ch, skin, face

# apply masks and transformations to target's latent space
def apply_transforms(target_image, mel_aged, oxy_aged, skin, face):
    app = SkinParameterAdjustmentApp(image=target_image, mel_aged=mel_aged, oxy_aged=oxy_aged,skin=skin,face=face,oxy_mask=oxy_mask)
    app.run()

if __name__ == '__main__':
    working_dir = os.getcwd()
    example_texture_path = textures_directory +"/m32_8k.png"
    target_texture_path = "textures/1_neutral.jpg"
    target_texture_path = textures_directory + "/1_neutral.jpg"
    target_texture_path = "textures/m53_4k.png"
    target_texture_path = textures_directory + "/m53_4k.png"
    warped_example_image, target_image, example_image = morph_images(Path(example_texture_path), Path(target_texture_path))
    Cm, Bh, skin, face = extract_masks(warped_example_image)
    apply_transforms(target_image, Cm, Bh, skin, face, oxy_mask)



max image 210.0
min image 0.0
max image 0.8235294222831726
min image 0.0
min Cm: 0.11022113263607025 max Cm: 0.5266589522361755
min Cm: 0.0 max Cm: 1.0
min Ch: 0.0 max Ch: 1.0
